In [ ]:
# Core scientific stack
import numpy as np
import matplotlib.pyplot as plt

# circle_bundles core API 
from circle_bundles.api import build_bundle

# local analysis
from circle_bundles.analysis.local_analysis import get_local_rips, plot_local_rips
from circle_bundles.analysis.fiberwise_clustering import (
    fiberwise_clustering, 
    plot_fiberwise_pca_grid,
    plot_fiberwise_summary_bars
)

#For computing local circular coordinates
from dreimac import CircularCoords

# Cover constructions
from circle_bundles.covers.triangle_cover_builders_fibonacci import (
    make_rp2_fibonacci_star_cover, 
    make_s2_fibonacci_star_cover
)

from circle_bundles.base_covers import MetricBallCover
from circle_bundles.metrics import S1AngleMetric as s1_metric

# Geometric signal unwrapping
from circle_bundles.geometry.geometric_unwrapping import lift_base_points, get_cocycle_dict
from circle_bundles.geometry.z2_linear import solve_Z2_edge_coboundary

# Synthetic data generation
from circle_bundles.synthetic.so3_sampling import sample_so3
from circle_bundles.synthetic.meshes import make_star_pyramid
from circle_bundles.synthetic.densities import (
    mesh_to_density,
    rotate_density,
    get_mesh_sample,
    get_density_axes
)

from circle_bundles.synthetic.mesh_vis import (
    make_density_visualizer,
    make_star_pyramid_visualizer,
)

# Visualization helpers (non-interactive, lightweight)
from circle_bundles.viz.thumb_grids import show_data_vis
from circle_bundles.viz.lattice_vis import lattice_vis

# Generate A Synthetic Dataset

In [ ]:
# --- Sanity check: generate one prism density + visualize a few SO(3) rotations ---

p = 5
height = 1
grid_size = 32
sigma = 0.05

mesh = make_star_pyramid(n_points = p, height = height)
density = mesh_to_density(mesh, grid_size=grid_size, sigma=sigma)

vis_density = make_density_visualizer(grid_size = grid_size)
vis_mesh = make_star_pyramid_visualizer(mesh)

n_samples = 8
rng = np.random.default_rng(0)
so3_data = sample_so3(n_samples=n_samples, rng=rng)[0]  

density_sample = rotate_density(density, so3_data, grid_size = grid_size)
fig = show_data_vis(density_sample, 
                       vis_func, 
                       max_samples = n_samples, 
                       n_cols = n_samples, 
                       sampling_method = 'first')
plt.show()

mesh_sample = get_mesh_sample(mesh, so3_data)

fig = show_data_vis(mesh_sample, 
                       mesh_vis, 
                       max_samples = n_samples, 
                       n_cols = n_samples, 
                       pad_frac = 0.3, 
                       sampling_method = 'first')
plt.show()


In [ ]:
# --- Generate the dataset ---

n_samples = 5000
rng = np.random.default_rng(0)
so3_data = sample_so3(n_samples=n_samples, rng=rng)[0]  

data = rotate_density(density, so3_data, grid_size = grid_size)
mesh_data = get_mesh_sample(mesh, so3_data)

print(
    f"Generated {n_samples} SO(3)-rotated star pyramid densities "
    f"represented as {data.shape[1]}-dimensional voxel vectors."
)



In [ ]:
# --- Compute base projections ---

base_points = get_density_axes(data)
print("Base projection coordinates computed.")


# Bundle Analysis

## Open Cover 

In [ ]:
# --- Construct an open cover of RP2 ---

n_landmarks = 40
rp2_cover = make_rp2_fibonacci_star_cover(base_points, n_pairs = n_landmarks)

summ = rp2_cover.summarize(plot = True)

## Fiberwise Clustering 

In [ ]:
#Run persistence on fibers to get an epsilon value for fiberwise clustering

fiber_ids, dense_idx_list, rips_list = get_local_rips(
    data,
    rp2_cover.U,
    p_values=None,
    to_view = [0,17,29],
    maxdim=1,
    n_perm=500,
    random_state=None,
)

fig, axes = plot_local_rips(
    fiber_ids,
    rips_list,
    n_cols=3,
    titles='default',
    font_size=20,
)

In [ ]:
#Run fiberwise clustering to separate each fiber into two components


eps_values = 0.0125*np.ones(len(rp2_cover.U))
min_sample_values = 5*np.ones(len(rp2_cover.U))

print('Running fiberwise clustering on the dataset...')
print('')
components, G, graph_dict, cl, summary = fiberwise_clustering(data, 
                                                              rp2_cover.U, 
                                                              eps_values, 
                                                              min_sample_values)

to_view = [0,1,2]
fig,ax = plot_fiberwise_pca_grid(summary, to_view = to_view)
plt.show()
fig, ax = plot_fiberwise_summary_bars(summary, hide_biggest=False)
plt.show()

In [ ]:
#Confirm that the Sigma_2 monodromy is non-trivial (i.e., the whole dataset is one connected component)


signs = get_cocycle_dict(G)

is_a_coboundary = solve_Z2_edge_coboundary(rp2_cover.nerve_edges(), signs, len(rp2_cover.U))[0]

print(f'Is a coboundary: {is_a_coboundary}')

## Lift To A Bundle Over $\mathbb{S}^{2}$ 

In [ ]:
#Construct a lift of the base map from RP2 to S2

lifted_base_points = lift_base_points(G, cl, base_points)

n_landmarks = 80
s2_cover = make_s2_fibonacci_star_cover(lifted_base_points, n_vertices = n_landmarks)
summ = s2_cover.summarize(plot = True)

print('Cover constructed.')

In [ ]:
#Compute local trivializations and characteristic classes

s2_bundle = build_bundle(
    data,
    s2_cover,
    CircularCoords_cls=CircularCoords,      #use sparse cc's for local circular coordinates
    show=True
)


In [ ]:
#Compute class persistence on the weights filtration of the nerve
pers = s2_bundle.get_persistence(show = True)


## Coordinate Bundle

In [ ]:
#Construct a classifying map to the Stiefel manifold and compute the pullback bundle
pullback_results = s2_bundle.get_pullback_data(
    subcomplex = 'full',
    base_weight=1.0,
    fiber_weight=1.0,
    packing = 'coloring2',
    show_summary = True,
)




In [ ]:
#Construct a pullback coordinate bundle object and verify it has the correct classification

pb_bundle = build_bundle(
    pullback_results.total_data,
    s2_cover,
#    CircularCoords_cls=CircularCoords,     # optionally use sparse cc's, but PCA2 is sufficient
    show=True,
    total_metric = pullback_results.metric 
)


# Restriction To The Equator $\mathbb{S}^{1}\subset \mathbb{S}^{2}$


In [ ]:
# --- Restrict to the S^1 "equator" in S^2 ---

eps = 0.15  # thickness of equatorial band

# Points near the equator: last coordinate close to 0
eq_mask = np.abs(s2_cover.base_points[:, -1]) < eps

eq_data = s2_bundle.data[eq_mask]
eq_mesh_data = mesh_data[eq_mask]

# Parametrize the equator by an angle in RP^1 (theta ~ theta + pi)
eq_base_angles = np.arctan2(s2_cover.base_points[eq_mask, 1], s2_cover.base_points[eq_mask, 0]) % (2*np.pi)

print(f"Equator band: {eq_data.shape[0]} / {s2_bundle.data.shape[0]} samples (eps={eps}).")

In [ ]:
#Construct a new bundle along the equator in RP2

#Set up an open cover of the base circle
n_landmarks = 12
landmarks = np.linspace(0, 2*np.pi, n_landmarks, endpoint= False)
overlap = 1.4
radius = overlap* np.pi/n_landmarks


eq_cover = MetricBallCover(eq_base_angles, landmarks, radius, metric = s1_metric())
eq_cover_data = eq_cover.build()

#Show a summary of the construction
eq_summ = eq_cover.summarize(plot = True)

In [ ]:
#Construct local circular coordinates and model transitions as O(2) matrices
eq_bundle = build_bundle(
    eq_data,
    eq_cover,
    CircularCoords_cls=CircularCoords,      
    show=True,
)


In [ ]:
#Compute global coordinates on equator data using a filtration of the nerve
eq_triv_result = eq_bundle.get_global_trivialization()
print('Global coordinates computed.')

In [ ]:
#Get a visualization of the data arranged by assigned coordinates using meshes
coords = np.array([eq_base_angles, eq_triv_result.F]).T
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(22, 10), dpi=200)

fig = lattice_vis(
    eq_mesh_data,
    coords,
    vis_mesh, 
    per_row=7,          
    per_col=7,          
    figsize=10, 
    thumb_px=100,
    dpi=200,
    ax = ax1,
)

#Get a visualization of the data arranged by assigned coordinates using meshes
fig = lattice_vis(
    eq_data,
    coords,
    vis_density, 
    per_row=7,          
    per_col=7,          
    figsize=11, 
    thumb_px=120,
    dpi=200,
    ax = ax2,
)
plt.show()


# Additional Visualizations

In [ ]:
#Show a visualization of a 'fat fiber' of the projection map

from circle_bundles.viz.fiber_vis import fiber_vis
from circle_bundles.viz.base_vis import base_vis

center_ind = 579
r = 0.2
dist_mat = rp2_cover.metric.pairwise(X=base_points)
nearby_indices = np.where(dist_mat[center_ind] < r)[0]

fiber_data = data[nearby_indices]
vis_data = mesh_data[nearby_indices]


fig = plt.figure(figsize=(18, 6), dpi=120)
ax1 = fig.add_subplot(1, 3, 1, projection="3d")
ax2 = fig.add_subplot(1, 3, 2, projection="3d")
ax3 = fig.add_subplot(1, 3, 3, projection="3d")

# PCA labeled with meshes
fiber_vis(
    fiber_data,
    mesh_vis,
    vis_data=vis_data,
    max_images=50,
    zoom=0.08,
    ax=ax1,
    show=False,
)
ax1.set_title("Fiber PCA (Meshes)")

# PCA labeled with density projections
fiber_vis(
    fiber_data,
    vis_func=vis_func,
    max_images=50,
    zoom=0.05,
    ax=ax2,
    show=False,
)
ax2.set_title("Fiber PCA (Densities)")

# Base visualization
base_vis(
    base_points,
    center_ind,
    r,
    dist_mat,
    use_pca=False,
    ax=ax3,
    show=False,
)
ax3.set_title("Base neighborhood")

plt.tight_layout()
plt.show()


In [ ]:
#Show the 1-skeleton of the nerve of the cover labeled by the permutation cocycle
from circle_bundles.viz.nerve_vis import nerve_vis

signs_O1 = {edge:(-1) ** signs[edge] for edge in signs.keys()}

dist_mat = rp2_cover.metric.pairwise(X = rp2_cover.landmarks, Y = rp2_cover.base_points)

node_labels = [f"{i+1}" for i in range(rp2_cover.landmarks.shape[0])]

fig, axes = nerve_vis(
    rp2_cover,
    cochains={1:signs_O1},
    base_colors={0:'black', 1:'black', 2:'pink'},
    cochain_cmaps={1:{1: 'blue', -1:'darkred'}},
    opacity=0,
    node_size=18,
    line_width=2,
    node_labels=node_labels,
    fontsize=9,
    font_color='lavender',
    title='1-Skeleton Of The Nerve Of The Cover'
)

plt.show()

In [ ]:
# Show the nodes of G labeled with + and - side by side

fig, axes = plt.subplots(1, 2, figsize=(14, 6), constrained_layout=True)

for ax, g in zip(axes, [0, 1]):

    sample_inds = []

    # Choose a representative for each cluster with label g
    for node in G.nodes():
        (j, k) = node
        if k == g:
            node_inds = cl[j] == k
            min_idx_local = np.argmin(dist_mat[j, node_inds])
            min_index = np.where(node_inds)[0][min_idx_local]
            sample_inds.append(min_index)

    sample_inds = np.array(sample_inds, dtype=int)

    sample_mesh_data = mesh_data[sample_inds]
    sign = "+" if g == 0 else "-"
    nerve_vis(
        rp2_cover,
        cochains={1: signs_O1},
        base_colors={0: "black", 1: "black", 2: "pink"},
        cochain_cmaps={1: {1: "blue", -1: "lightgray"}},
        node_size=30,
        line_width=1,
        node_labels=None,
        fontsize=16,
        font_color="white",
        vis_func=mesh_vis,
        data=sample_mesh_data,
        image_zoom=0.105,
        title = rf"$X^{{{sign}}}$ Clusters",
        ax=ax,                
        show=False,           
    )

plt.show()


In [ ]:
#Attach extra visualization methods to bundle objects

from circle_bundles.bundle import attach_bundle_viz_methods

attach_bundle_viz_methods()


In [ ]:
#Show some correlations between local circular coordinates on overlaps
fig = s2_bundle.compare_trivs(align = True, ncols = 3)
plt.show()

In [ ]:
#Show a visualization of the restricted nerve
fig = s2_bundle.show_max_trivial()
plt.show()

In [ ]:
#Show a visualization of the nerve of the cover of the equator

#Compute a potential for the restricted orientation class
eq_subcomplex = eq_bundle.get_max_trivial_subcomplex()
edges = eq_subcomplex.kept_edges
Omega = eq_bundle.classes.cocycle_used.restrict(edges)
phi_vec = Omega.orient_if_possible(edges)[2]
phi = {lmk: phi_vec[lmk] for lmk in range(n_landmarks)}
omega = eq_bundle.classes.omega_O1_used

fig = eq_bundle.show_circle_nerve(omega = omega, phi = phi)
plt.show()


